# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [1]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('./CONLL2003/CONLL2003', 'train.txt', ['words', 'pos', 'ignore', 'chunk'])
training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
   a_dict = {
      "word.lower()": token.lower(),
      "word.isupper()": token.isupper(),
      "word.istitle()": token.istitle(),
      "word.isdigit()": token.isdigit(),
      "postag": pos,
      "word.shape": ''.join(['X' if c.isupper() else 'X' if c.islower() else 'd' if c.isdigit() else '-' for c in token])
   }
   training_features.append(a_dict)
   training_gold_labels.append(ne_label)  

In [2]:
### Adapt the path to point to the CONLL2003 folder on your local machine
test = ConllCorpusReader('./CONLL2003/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

test_features = []
test_gold_labels = []
for token, pos, ne_label in test.iob_words():
    a_dict = {
        "word.lower()": token.lower(),
        "word.isupper()": token.isupper(),
        "word.istitle()": token.istitle(),
        "word.isdigit()": token.isdigit(),
        "postag": pos,
        "word.shape": ''.join(['X' if c.isupper() else 'X' if c.islower() else 'd' if c.isdigit() else '-' for c in token])
    }
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)


**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [3]:
from collections import Counter 

my_list=[1,2,1,3,2,5]
Counter(my_list)


Counter({1: 2, 2: 2, 3: 1, 5: 1})

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [ ]:
# %pip install --upgrade numpy==1.26.4
from sklearn.feature_extraction import DictVectorizer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
vec = DictVectorizer()

# Concatenate train and test features
combined_features = training_features + test_features

# Initialize the DictVectorizer
vec = DictVectorizer()

# Fit the vectorizer on the combined features
the_array = vec.fit_transform(combined_features)

# Store the number of training instances to use as the split index
num_train_instances = len(training_features)

# Split the array back into training and test sets using the index
X_train_vectorized = the_array[:num_train_instances]
X_test_vectorized = the_array[num_train_instances:]

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?

The classifier performs well on the 'O' label because it is the most frequent label.
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?
The classifier might perform poorly on less frequent labels like 'I-LOC', 'I-MISC' due to insufficient training examples.

<i>If you run with the complete CONLL data, you will get a more detailed report that allows proper analysis of the model's strengths and weaknesses.</i>

In [7]:
from sklearn import svm

In [8]:
lin_clf = svm.LinearSVC()

In [9]:
# Train the classifier
from sklearn.metrics import classification_report


lin_clf.fit(X_train_vectorized, training_gold_labels)

# Make predictions on the test data
y_pred = lin_clf.predict(X_test_vectorized)

# Generate the classification report
report = classification_report(test_gold_labels, y_pred)

# Print the classification report
print(report)

c:\dev\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

       B-LOC       0.78      0.79      0.79      1668
      B-MISC       0.72      0.69      0.70       702
       B-ORG       0.72      0.57      0.64      1661
       B-PER       0.65      0.58      0.61      1617
       I-LOC       0.64      0.56      0.60       257
      I-MISC       0.63      0.58      0.61       216
       I-ORG       0.66      0.49      0.56       835
       I-PER       0.43      0.62      0.51      1156
           O       0.98      0.99      0.99     38323

    accuracy                           0.93     46435
   macro avg       0.69      0.65      0.67     46435
weighted avg       0.93      0.93      0.93     46435



**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [ ]:
# your code here

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [ ]:
import pandas

In [ ]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/Users/piek/Desktop/ONDERWIJS/data/nerc_datasets/kaggle/ner_v2.csv'
kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook